<a href="https://colab.research.google.com/github/MovinduSenura/Chest-Xray-Pneumonia-Detection/blob/dev-senura/notebooks/CustomCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# IMPORTS

import os
import sys
import importlib
from google.colab import drive
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization

In [4]:
# CLONE GITHUB REPOSITORY

if not os.path.exists('Chest-Xray-Pneumonia-Detection'):
    !git clone https://github.com/MovinduSenura/Chest-Xray-Pneumonia-Detection.git

In [5]:
# ADD NOTEBOOKS FOLDER TO PYTHON PATH

sys.path.append('/content/Chest-Xray-Pneumonia-Detection/notebooks/')

In [6]:
# IMPORT AND RELOAD PREPROCESSING MODULE

import preprocessing
importlib.reload(preprocessing)    # reloads latest version
from preprocessing import create_generators

In [7]:
# MOUNT GOOGLE DRIVE (DATASET LOCATION)

drive.mount('/content/drive')
DATA_DIR = '/content/drive/MyDrive/DL_Assignment/chest_xray/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
# CREATE TRAIN/VAL/TEST GENERATORS

train_gen, val_gen, test_gen = create_generators(DATA_DIR)
print("✅ Data generators created successfully!\n")

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
✅ Data generators created successfully!



In [9]:
# BUILD CUSTOM CNN MODEL

model = Sequential([
    # Convolutional Block 1
    Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3)),
    BatchNormalization(),
    MaxPooling2D(2,2),

    # Convolutional Block 2
    Conv2D(64, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),

    # Convolutional Block 3
    Conv2D(128, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),
    Dropout(0.3),

    # Fully Connected Layers
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 222, 222, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 109, 109, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 52, 52, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,169,985 (42.61 MB)

 Trainable params: 11,169,537 (42.61 MB)

 Non-trainable params: 448 (1.75 KB)

In [10]:
# TRAIN THE MODEL

history = model.fit(
    train_gen,
    epochs=5,
    validation_data=val_gen
)

Epoch 1/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 120s 701ms/step - accuracy: 0.7840 - loss: 2.7066 - val_accuracy: 0.6250 - val_loss: 9.1833
Epoch 2/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 114s 699ms/step - accuracy: 0.8875 - loss: 0.2705 - val_accuracy: 0.5000 - val_loss: 25.1314
Epoch 3/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 114s 700ms/step - accuracy: 0.9159 - loss: 0.2060 - val_accuracy: 0.5000 - val_loss: 5.4232
Epoch 4/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 113s 696ms/step - accuracy: 0.8968 - loss: 0.2454 - val_accuracy: 0.7500 - val_loss: 0.6082
Epoch 5/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 112s 688ms/step - accuracy: 0.9255 - loss: 0.1907 - val_accuracy: 0.5625 - val_loss: 0.7626


In [11]:
# EVALUATE MODEL PERFORMANCE

test_loss, test_acc = model.evaluate(test_gen)
print(f"\n✅ Test Accuracy: {test_acc:.3f}")
print(f"✅ Test Loss: {test_loss:.3f}")

20/20 ━━━━━━━━━━━━━━━━━━━━ 147s 8s/step - accuracy: 0.8801 - loss: 0.3483

✅ Test Accuracy: 0.832
✅ Test Loss: 0.445
